Install Libraries

In [ ]:
# %pip install numpy
# %pip install pandas
# %pip install matplotlib
# %pip install seaborn
%pip install -U scikit-learn

Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

Import Dataset

In [ ]:
df = pd.read_csv('dataset.csv')
# df.head(15)

Data-Preprocessing step
  - Omit is_ne column
    - Will lead to oversampling, because of a small amount of data. Too specifc, needs to be more general
  - Omit rows that have null values under the 'word' column
    - Unnecessary data

In [ ]:
original_df = df
# print(len(original_df))
is_ne_drop = df.drop("is_ne", axis=1)

# for (i in range ):
word_null_query = is_ne_drop[is_ne_drop['word'].isnull()]

word_null_query.head()

# for word_null_query in original_df:
# word_null_query.head(16)

filtered_df = is_ne_drop.drop(word_null_query.index)
# print(len(original_df))
filtered_df.head()


Feature Engineering / Preprocessing:

Target Features:
 - Word length
 - Vowel / Word Ratio
 - Consonant / Word Ratio
 - Look at common letters not present in filipino word (c, x, z, f, etc.)
 - Common NGram counts for Filipino and English words
    - ng
    - ch, sh, etc.
    - Filipino prefixes (um, in, etc.)
-  Check if the word is a (noun, verb, etc based on the sentence)


Note: normalize data if needed (check sklearn.preprocessing)

Additional Features:
- Special character handling for words classfied as OTH

In [ ]:
len(df["is_spelling_correct"])

Length Feature

In [ ]:
df['word_length'] = df['word'].apply(lambda w: len(str(w)) if isinstance(w, str) else 0)

Vowel and consonant ratio

In [ ]:
def vowel_consonant_ratio(word):
    if not isinstance(word, str):  ### if the word does not have alphabet
        return 0.0
    
    vowels = ['a', 'e', 'i', 'o', 'u', 'A', 'E', 'I', 'O', 'U']
    num_vowels = sum(1 for ch in word if ch.isalpha() and ch in vowels)
    num_consonants = sum(1 for ch in word)

    if num_consonants == 0:
        return 1.0 if num_vowels > 0 else 0.0
    return num_vowels / num_consonants

def vowel_word_ratio(word):
    if not isinstance(word, str):
        return 0.0
    
    vowels = ['a', 'e', 'i', 'o', 'u', 'A', 'E', 'I', 'O', 'U']
    num_vowels = sum(1 for ch in word if ch.isalpha() and ch in vowels)
    num_consonants = sum(1 for ch in word)

    if num_consonants == 0:
        return 1.0 if num_vowels > 0 else 0.0
    return num_vowels / num_consonants

df['vowel_word_ratio'] = df['word'].apply(vowel_word_ratio)
df['vowel_consonant_ratio'] = df['word'].apply(vowel_consonant_ratio)

Imbalance Data:

Check these techniques in imblearn library:
 - SMOTE
 - Undersampling
 - Oversampling

Train Test Split

- Check the sklearn to split.
- Model should be 80-20 split

ML Modelling

Naive Bayes

Decision Trees

Model Validation:
- Confusion Matrix
- Classification report